In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session#

In [1]:
#import libraries 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
import cv2
import numpy as np
import os
from sklearn.metrics import precision_score, recall_score, jaccard_score
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import CLIPModel, CLIPProcessor
import glob


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
2025-05-11 11:19:16.605716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746962356.824026      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746962356.889743      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# CBAM Implementation
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_planes, ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x


In [3]:
# Configurations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 2
lr = 1e-3  # Updated learning rate
num_epochs = 200
edge_methods = ['sobel', 'canny', 'laplacian']
vlm_feature_size = 512
lambda_dice = 1.0  # Weight for Dice loss


In [4]:
# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [5]:
#extracting VLM features
def extract_vlm_features(image):
    if image.ndim == 2:
        image = np.stack([image]*3, axis=-1)
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
    return features.cpu().numpy()


In [6]:
# Combined edge map
def get_edge_map(image, methods=['sobel', 'canny', 'laplacian']):
    edge_maps = []
    for method in methods:
        if method == 'canny':
            edge_map = cv2.Canny(image, 50, 150)
        elif method == 'sobel':
            sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
            sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
            edge_map = np.sqrt(sobelx**2 + sobely**2)
        elif method == 'laplacian':
            edge_map = cv2.Laplacian(image, cv2.CV_64F, ksize=3)
        edge_maps.append(edge_map)
    combined_edge_map = np.mean(np.array(edge_maps), axis=0)
    combined_edge_map = np.abs(combined_edge_map)
    combined_edge_map = (combined_edge_map - combined_edge_map.min()) / (combined_edge_map.max() - combined_edge_map.min() + 1e-8)
    return combined_edge_map.astype(np.float32)


In [7]:
#preprocessing

# Transforms
train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(256, 256),
    ToTensorV2()
])


In [8]:
# Dataset Class
class TNBCDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None, edge_methods=['sobel']):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform
        self.edge_methods = edge_methods

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx], cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(self.mask_paths[idx], cv2.IMREAD_GRAYSCALE)
        edges = get_edge_map(image, self.edge_methods)
        combined_image = np.stack([image, edges], axis=-1)
        image_rgb = np.stack([image]*3, axis=-1)
        vlm_features = extract_vlm_features(image_rgb)

        if self.transform:
            augmented = self.transform(image=combined_image, mask=mask)
            combined_image = augmented['image']
            mask = augmented['mask'].float() / 255.0
        else:
            combined_image = torch.from_numpy(combined_image.transpose(2, 0, 1)).float()
            mask = torch.from_numpy(mask).float() / 255.0

        if mask.dim() == 2:
            mask = mask.unsqueeze(0)

        vlm_features = torch.tensor(vlm_features, dtype=torch.float32).squeeze(0)
        return combined_image, mask, vlm_features

    def __len__(self):
        return len(self.image_paths)


In [9]:
# Model Architecture
class AttentionUNet(nn.Module):
    def __init__(self, in_channels=2, out_channels=1, vlm_feature_size=512):
        super().__init__()
        self.enc1 = self.conv_block(in_channels, 64)
        self.cbam1 = CBAM(64)
        self.enc2 = self.conv_block(64, 128)
        self.cbam2 = CBAM(128)
        self.enc3 = self.conv_block(128, 256)
        self.cbam3 = CBAM(256)
        self.enc4 = self.conv_block(256, 512)
        self.cbam4 = CBAM(512)
        self.pool = nn.MaxPool2d(2)

        self.vlm_proj1 = nn.Sequential(
            nn.Linear(vlm_feature_size, 64),
            nn.Unflatten(1, (64, 1, 1)))
        self.vlm_proj2 = nn.Sequential(
            nn.Linear(vlm_feature_size, 128),
            nn.Unflatten(1, (128, 1, 1)))
        self.vlm_proj3 = nn.Sequential(
            nn.Linear(vlm_feature_size, 256),
            nn.Unflatten(1, (256, 1, 1)))
        self.vlm_proj4 = nn.Sequential(
            nn.Linear(vlm_feature_size, 512),
            nn.Unflatten(1, (512, 1, 1)))

        self.dec1 = self.upconv_block(512, 256)
        self.att1 = AttentionBlock(256, 256, vlm_dim=256)
        self.dec2 = self.upconv_block(512, 128)
        self.att2 = AttentionBlock(128, 128, vlm_dim=128)
        self.dec3 = self.upconv_block(256, 64)
        self.att3 = AttentionBlock(64, 64, vlm_dim=64)
        self.final = nn.Conv2d(128, out_channels, 1)

    def conv_block(self, in_c, out_c):
        return nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def upconv_block(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 2, 2),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )

    def forward(self, x, vlm_features):
        vlm1 = self.vlm_proj1(vlm_features)
        vlm2 = self.vlm_proj2(vlm_features)
        vlm3 = self.vlm_proj3(vlm_features)
        vlm4 = self.vlm_proj4(vlm_features)

        e1 = self.enc1(x)
        e1 = self.cbam1(e1)
        e2 = self.enc2(self.pool(e1))
        e2 = self.cbam2(e2)
        e3 = self.enc3(self.pool(e2))
        e3 = self.cbam3(e3)
        e4 = self.enc4(self.pool(e3))
        e4 = self.cbam4(e4)

        d1 = self.dec1(e4)
        a1 = self.att1(d1, e3, vlm3)
        d1 = torch.cat([a1, d1], dim=1)

        d2 = self.dec2(d1)
        a2 = self.att2(d2, e2, vlm2)
        d2 = torch.cat([a2, d2], dim=1)

        d3 = self.dec3(d2)
        a3 = self.att3(d3, e1, vlm1)
        d3 = torch.cat([a3, d3], dim=1)

        return torch.sigmoid(self.final(d3))

#Attention module

class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, vlm_dim):
        super().__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_l, 1),
            nn.BatchNorm2d(F_l)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l + vlm_dim, F_l, 1),
            nn.BatchNorm2d(F_l)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_l, 1, 1),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU()

    def forward(self, g, x, vlm):
        vlm_spatial = F.interpolate(vlm, size=x.shape[2:], mode='bilinear')
        x_vlm = torch.cat([x, vlm_spatial], dim=1)
        g1 = self.W_g(g)
        x1 = self.W_x(x_vlm)
        if g1.shape[2:] != x1.shape[2:]:
            g1 = F.interpolate(g1, size=x1.shape[2:], mode='bilinear')
        psi = self.relu(g1 + x1)
        return x * self.psi(psi)

#Discriminator

class Discriminator(nn.Module):
    def __init__(self, in_channels=2):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 4, 1, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 1, 4, 1, 1)
        )

    def forward(self, x):
        return self.model(x)


In [10]:
# Initialize models and optimizers
generator = AttentionUNet(vlm_feature_size=vlm_feature_size).to(device)
discriminator = Discriminator().to(device)

# Loss functions
bce_loss = nn.BCEWithLogitsLoss()

def dice_loss(pred, target, smooth=1.0):
    pred = pred.contiguous().view(pred.shape[0], -1)
    target = target.contiguous().view(target.shape[0], -1)
    intersection = (pred * target).sum(dim=1)
    dice = (2. * intersection + smooth) / (pred.sum(dim=1) + target.sum(dim=1) + smooth)
    return 1 - dice.mean()

opt_g = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
opt_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))


In [11]:
# Metrics and Data Preparation
def calculate_metrics(pred, target):
    pred_bin = (pred > 0.5).float()
    target = target.cpu().numpy().astype(np.uint8)
    pred_bin = pred_bin.cpu().numpy().astype(np.uint8)
    dice = (2 * (pred_bin * target).sum()) / (pred_bin.sum() + target.sum() + 1e-8)
    iou = jaccard_score(target.flatten(), pred_bin.flatten(), average='binary')
    precision = precision_score(target.flatten(), pred_bin.flatten(), average='binary', zero_division=0)
    recall = recall_score(target.flatten(), pred_bin.flatten(), average='binary')
    return dice, iou, precision, recall


In [12]:
# Dataset Paths
train_image_dir = "/kaggle/input/ph2segmentation/Train/Images"
train_mask_dir = "/kaggle/input/ph2segmentation/Train/Masks"
val_image_dir = "/kaggle/input/ph2segmentation/Validation/Images"
val_mask_dir = "/kaggle/input/ph2segmentation/Validation/Masks"
test_image_dir = "/kaggle/input/ph2segmentation/Test/Images"
test_mask_dir = "/kaggle/input/ph2segmentation/Test/Masks"

train_image_paths = sorted(glob.glob(os.path.join(train_image_dir, "*")))
train_mask_paths = sorted(glob.glob(os.path.join(train_mask_dir, "*")))
val_image_paths = sorted(glob.glob(os.path.join(val_image_dir, "*")))
val_mask_paths = sorted(glob.glob(os.path.join(val_mask_dir, "*")))
test_image_paths = sorted(glob.glob(os.path.join(test_image_dir, "*")))
test_mask_paths = sorted(glob.glob(os.path.join(test_mask_dir, "*")))

train_dataset = TNBCDataset(train_image_paths, train_mask_paths, train_transform, edge_methods)
val_dataset = TNBCDataset(val_image_paths, val_mask_paths, val_transform, edge_methods)
test_dataset = TNBCDataset(test_image_paths, test_mask_paths, val_transform, edge_methods)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size)
test_loader = DataLoader(test_dataset, batch_size)


In [ ]:
# Training, Validation, Testing

# Training Loop
best_val_dice = 0
for epoch in range(num_epochs):
    generator.train()
    discriminator.train()
    running_metrics = {'dice': 0, 'iou': 0, 'precision': 0, 'recall': 0}

    for images, masks, vlm_features in tqdm(train_loader):
        images, masks = images.to(device), masks.to(device)
        vlm_features = vlm_features.to(device)

        if masks.dim() == 3:
            masks = masks.unsqueeze(1)

        # Train Discriminator
        opt_d.zero_grad()

        # Real samples
        real_input = torch.cat([images[:, :1, :, :], masks], dim=1)
        real_pred = discriminator(real_input)
        real_loss = bce_loss(real_pred, torch.ones_like(real_pred))

        # Fake samples
        with torch.no_grad():
            fake_masks = generator(images, vlm_features)
        fake_input = torch.cat([images[:, :1, :, :], fake_masks], dim=1)
        fake_pred = discriminator(fake_input.detach())
        fake_loss = bce_loss(fake_pred, torch.zeros_like(fake_pred))

        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        opt_d.step()
        # Train Generator
        opt_g.zero_grad()
        fake_masks = generator(images, vlm_features)

        # Dice Loss
        dice = dice_loss(fake_masks, masks)

        # Adversarial loss
        fake_input = torch.cat([images[:, :1, :, :], fake_masks], dim=1)
        g_pred = discriminator(fake_input)
        g_loss = bce_loss(g_pred, torch.ones_like(g_pred))

        # Total generator loss
        total_loss = g_loss + lambda_dice * dice
        total_loss.backward()
        opt_g.step()

        # Calculate metrics
        with torch.no_grad():
            dice, iou, prec, rec = calculate_metrics(fake_masks, masks)
            running_metrics['dice'] += dice
            running_metrics['iou'] += iou
            running_metrics['precision'] += prec
            running_metrics['recall'] += rec
    # Validation
    generator.eval()
    val_metrics = {'dice': 0, 'iou': 0, 'precision': 0, 'recall': 0}
    with torch.no_grad():
        for images, masks, vlm_features in val_loader:
            images, masks = images.to(device), masks.to(device)
            vlm_features = vlm_features.to(device)
            fake_masks = generator(images, vlm_features)
            dice, iou, prec, rec = calculate_metrics(fake_masks, masks)
            val_metrics['dice'] += dice
            val_metrics['iou'] += iou
            val_metrics['precision'] += prec
            val_metrics['recall'] += rec

    val_dice = val_metrics['dice'] / len(val_loader)
    if val_dice > best_val_dice:
        best_val_dice = val_dice
        torch.save(generator.state_dict(), 'best_generator.pth')

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Dice: {running_metrics['dice']/len(train_loader):.4f}")
    print(f"Val Dice: {val_dice:.4f}")



In [ ]:
# Testing
generator.load_state_dict(torch.load('best_generator.pth'))
generator.eval()
test_metrics = {'dice': 0, 'iou': 0, 'precision': 0, 'recall': 0}

with torch.no_grad():
    for images, masks, vlm_features in test_loader:
        images, masks = images.to(device), masks.to(device)
        vlm_features = vlm_features.to(device)
        fake_masks = generator(images, vlm_features)
        dice, iou, prec, rec = calculate_metrics(fake_masks, masks)
        test_metrics['dice'] += dice
        test_metrics['iou'] += iou
        test_metrics['precision'] += prec
        test_metrics['recall'] += rec

print("\nFinal Test Results:")
print(f"Dice: {test_metrics['dice']/len(test_loader):.4f}")
print(f"IoU: {test_metrics['iou']/len(test_loader):.4f}")
print(f"Precision: {test_metrics['precision']/len(test_loader):.4f}")
print(f"Recall: {test_metrics['recall']/len(test_loader):.4f}")